In [1]:
import cv2 as cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
labels = pd.read_csv(f"Data_Files/annotations.csv", sep=';')
# keep only rows where Err is NaN
labels = labels[labels['Err'].isna()]

In [3]:
labels['x'] = 0
labels['y'] = 0
labels['w'] = 0
labels['h'] = 0

In [4]:
labels.head(10)

,CID,Image,Gender,Ethnicity,LR,Err,x,y,w,h
0,421,1,m,1,r,NaN,0,0,0,0
1,421,2,m,1,l,NaN,0,0,0,0
2,421,3,m,1,r,NaN,0,0,0,0
4,421,5,m,1,l,NaN,0,0,0,0
5,421,6,m,1,l,NaN,0,0,0,0
6,421,7,m,1,l,NaN,0,0,0,0
7,421,8,m,1,r,NaN,0,0,0,0
8,421,9,m,1,l,NaN,0,0,0,0
9,421,10,m,1,l,NaN,0,0,0,0
10,421,11,m,1,r,NaN,0,0,0,0


In [5]:
for ix, row in labels.iterrows():
    # print(row['CID'])
    d = "%03d" % row['CID']
    f = "%02d" % row['Image']
    edged = cv2.imread(f"Data_Files/masks/{d}/{f}.png", cv2.IMREAD_GRAYSCALE)
    contours, hierarchy = cv2.findContours(edged,
                                       cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_NONE)
    x,y,w,h = cv2.boundingRect(contours[0])
    # set values where CIS is row['CID'] and Image is row['Image']
    labels.loc[(labels['CID'] == row['CID']) & (labels['Image'] == row['Image']), 'x'] = x
    labels.loc[(labels['CID'] == row['CID']) & (labels['Image'] == row['Image']), 'y'] = y
    labels.loc[(labels['CID'] == row['CID']) & (labels['Image'] == row['Image']), 'w'] = w
    labels.loc[(labels['CID'] == row['CID']) & (labels['Image'] == row['Image']), 'h'] = h

In [6]:
labels.head(10)

,CID,Image,Gender,Ethnicity,LR,Err,x,y,w,h
0,421,1,m,1,r,NaN,81,53,21,29
1,421,2,m,1,l,NaN,388,79,56,79
2,421,3,m,1,r,NaN,409,19,80,123
4,421,5,m,1,l,NaN,612,182,81,102
5,421,6,m,1,l,NaN,832,296,97,141
6,421,7,m,1,l,NaN,1089,312,120,176
7,421,8,m,1,r,NaN,234,158,44,68
8,421,9,m,1,l,NaN,558,443,145,126
9,421,10,m,1,l,NaN,1043,382,136,180
10,421,11,m,1,r,NaN,1048,667,220,290


In [7]:
# drop Err column
labels = labels.drop(columns=['Err'])

In [8]:
labels.to_csv('Data_Files/bounding_box_data.csv', index=False)

In [9]:
labels = pd.read_csv(f"Data_Files/bounding_box_data.csv")
for ix, row in labels.iterrows():
    # print(row['CID'])
    d = "%03d" % row['CID']
    f = "%02d" % row['Image']
    img = cv2.imread(f"Data_Files/images/{d}/{f}.png")
    x = row['x']
    y = row['y']
    w = row['w']
    h = row['h']
    crop_img = img[y:y+h, x:x+w]
    # create dir if it does not exist
    if not os.path.exists(f"Data_Files/cropped_images/{d}"):
        os.makedirs(f"Data_Files/cropped_images/{d}")

    cv2.imwrite(f"Data_Files/cropped_images/{d}/{f}.png", crop_img)